In [2]:
from keras.models import Sequential
from keras.layers import Lambda, Dense, GlobalAveragePooling2D
import tensorflow as tf
import keras.applications.mobilenet_v2 as mobilenetv2

In [3]:
# Image dimensions
IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS = 224, 224, 3

# Original Category mappings (12 categories)
categories = {0: 'paper', 1: 'cardboard', 2: 'plastic', 3: 'metal', 4: 'trash', 
              5: 'battery', 6: 'shoes', 7: 'clothes', 8: 'green-glass', 
              9: 'brown-glass', 10: 'white-glass', 11: 'biological'}

In [21]:
# The model's internal category indexing not align with the Original Category mappings
# Reverse engineer to find the correct classification label (Plastic) - using train set plastic image
def map_to_plastic_or_nonplastic(predicted_class):
    # 3 is Plastic
    if predicted_class == 3:
        return "Plastic"
    else:
        return "Non-Plastic"

In [5]:
import h5py

# Open the .h5 file
with h5py.File('waste_classification_pretrained_mobileNetv2.h5', 'r') as f:
    # Print all layer names and their weights
    for layer_name in f.keys():
        print(f"Layer name: {layer_name}")
        for weight_name in f[layer_name].keys():
            print(f"    Weight name: {weight_name}, shape: {f[layer_name][weight_name]}")


Layer name: dense
    Weight name: dense, shape: <HDF5 group "/dense/dense" (2 members)>
Layer name: global_average_pooling2d
Layer name: lambda
Layer name: mobilenetv2_1.00_224
    Weight name: Conv1, shape: <HDF5 group "/mobilenetv2_1.00_224/Conv1" (1 members)>
    Weight name: Conv_1, shape: <HDF5 group "/mobilenetv2_1.00_224/Conv_1" (1 members)>
    Weight name: Conv_1_bn, shape: <HDF5 group "/mobilenetv2_1.00_224/Conv_1_bn" (4 members)>
    Weight name: block_10_depthwise, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_depthwise" (1 members)>
    Weight name: block_10_depthwise_BN, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_depthwise_BN" (4 members)>
    Weight name: block_10_expand, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_expand" (1 members)>
    Weight name: block_10_expand_BN, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_expand_BN" (4 members)>
    Weight name: block_10_project, shape: <HDF5 group "/mobilenetv2_1.00_224/block_10_project" (1 members)

### Transfer Learning – MobileNetV2 weights are frozen, and only the classification head is trained for waste classification.

MobileNetV2 weights - https://www.kaggle.com/code/alexfordna/garbage-classification-mobilenetv2-92-accuracy/input?select=garbage_classification

In [6]:
# Load MobileNetV2 base model without the top layer
mobilenetv2_layer = mobilenetv2.MobileNetV2(include_top=False, 
                                            input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),
                                            weights='mobilenet-v2-keras-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5')

mobilenetv2_layer.trainable = False


# Create the model architecture
model = Sequential()
# Input layer
model.add(tf.keras.Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))

# Lambda preprocessing layer
def mobilenetv2_preprocessing(img):
    return mobilenetv2.preprocess_input(img)
model.add(Lambda(mobilenetv2_preprocessing))

model.add(mobilenetv2_layer)

# Add the GlobalAveragePooling2D layer
model.add(GlobalAveragePooling2D())

# Add the Dense layer for classification
model.add(Dense(12, activation='softmax'))

# Load the weights from your .h5 file
model.load_weights('waste_classification_pretrained_mobileNetv2.h5')

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 12)             │        15,372 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,273,356 (8.67 MB)

 Trainable params: 15,372 (60.05 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [14]:
if hasattr(model, 'class_indices'):
    print("Class indices found:", model.class_indices)
else:
    print("No class indices found in the model.")

No class indices found in the model.


In [7]:
import cv2
import numpy as np

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    
    # Resize the image to 224x224
    image_resized = cv2.resize(image, (224, 224))
    
    # Convert image to float32 and normalize using MobileNetV2's preprocess_input
    image_normalized = mobilenetv2.preprocess_input(image_resized.astype(np.float32))
    
    # Add a batch dimension (the model expects input shape: (batch_size, 224, 224, 3))
    image_batch = np.expand_dims(image_normalized, axis=0)
    
    return image_batch

In [8]:
def classify_image(model, image_path):
    # Preprocess the image
    image_batch = preprocess_image(image_path)
    
    # Perform inference
    predictions = model.predict(image_batch)
    
    # Get the predicted class (argmax - the index of the highest probability)
    predicted_class = np.argmax(predictions, axis=1)[0]
    
    # Map the predicted class to plastic or non-plastic before returning
    return map_to_plastic_or_nonplastic(predicted_class)

In [22]:
# image_path = 'plastic_bottle_out_of_shape.jpg'
# image_path = 'plastic_bottle_white_background.jpg'
image_path = 'plastic146_in_dataset.jpg' 
# image_path = 'plastic_bottle.jpg'

result = classify_image(model, image_path)

print(f"The object is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
The object is classified as: Plastic
